In [ ]:
"""
목표: dataclass로 인자의 기본값,선택지,설명,타입을 담고 설정하기 위한 클래스를 만들며 이를 추후 cli, gui로 확장.
추가로, whisperx나 ytdlp의 parser에서 arg들의 설명과 기본값, 타입 등을 가져와서 이 형태로 변환하는 것도 테스트
"""

In [2]:
from dataclasses import dataclass, asdict, field, fields
from typing import Optional, List, Literal
from rich.pretty import pprint


@dataclass
class AudioConfig:
    sample_rate: Literal[8000, 16000, 44100] = field(
        default=16000,
        metadata={
            "choices": [8000, 16000, 44100],
            "help": "Sampling rate (Hz)",
            "cli": "--sample-rate",
            "group": "Audio",
        }
    )

    frame_ms: float = field(
        default=25.0,
        metadata={
            "min": 5.0,
            "max": 100.0,
            "help": "Frame size in milliseconds",
            "cli": "--frame-ms",
            "group": "Audio",
        }
    )

    def __post_init__(self):
        pass



In [3]:

def foo(args: AudioConfig):
    args_dict = asdict(args)
    pprint(args_dict)

config = AudioConfig(sample_rate=8000)
foo(config)
# 결과: {'sample_rate': 16000, 'frame_ms': 25.0}



{'sample_rate': 8000, 'frame_ms': 25.0}

In [4]:
def read_and_convert(args: AudioConfig):
    """for문으로 값을 읽어오고 사용"""
    for f in fields(args):
        print(f.name)
        print(f.type) # choices 없이 그냥 이걸 파싱해서 쓰면 된다. 선택지가 런타입에 변하지만 않으면
        print(f.default)
        print(f.metadata)
        print(f.metadata.get("choices")) # 터지지 않고 메타데이터를 가져오는 방법

read_and_convert(config)



sample_rate
typing.Literal[8000, 16000, 44100]
16000
{'choices': [8000, 16000, 44100], 'help': 'Sampling rate (Hz)', 'cli': '--sample-rate', 'group': 'Audio'}
[8000, 16000, 44100]
frame_ms
<class 'float'>
25.0
{'min': 5.0, 'max': 100.0, 'help': 'Frame size in milliseconds', 'cli': '--frame-ms', 'group': 'Audio'}
None


In [6]:
def read_only_one(args: AudioConfig):
    """하나만 콕 집어서 읽어옴"""
    help = args.__dataclass_fields__["sample_rate"].metadata["help"]
    print(help)

read_only_one(config)


Sampling rate (Hz)


In [ ]:
# 상속을 통한 자동 함수 테스트
"""
BaseConfig를 만들어서 여기에 cli, gui 변환 함수를 만들고 상속.
@dataclass는 상속되지 않으므로 부모와 자식 모두에 @dataclass를 붙여야 함. 혹은 자식에만 붙임.
부모에 함수는 @classmethod로 만들어야 한다고 함.
"""
class BaseConfig:
    @classmethod
    def to_cli_args(cls):
        args = []
        for f in fields(cls):
            cli_flag = f.metadata.get("cli")
            if cli_flag:
                args.append(f"{cli_flag} {f.default}")
        return args
    
@dataclass
class WhisperConfig(BaseConfig):
    model_size: Literal["tiny", "base", "small", "medium", "large"] = field(
        default="small",
        metadata={
            "choices": ["tiny", "base", "small", "medium", "large"],
            "help": "Whisper model size",
            "cli": "--model-size",
            "group": "Whisper",
        }
    )


